# Data Colning and Data Transformation to MYSQL


# Steps in the Process:
 
01) Data Colning from Phonepe pulse data and storing it in Local Directory
02) Extracting the necessary data from cloned data and converting them into Dataframes
03) Performing cleaning process in the dataframe
04) Converting each dataframe to CSV Files 
05) Connecting to MYSQL workbench and creating a phonepe database
06) In Phonepe Databas inserting each CSV files as individual Tables

In [90]:
# [clone libraries]
import requests
import subprocess
# import git

# [pandas and file handling libraries]
import pandas as pd
import os
import json

# [SQL libraries]
import mysql.connector
import sqlalchemy
from sqlalchemy import create_engine

In [3]:
!git --version

git version 2.42.0.windows.2


In [4]:
!git --version


'C:\\Users\\DIVAHAR\\phonepe'

# Cloning process
01) Conecting to the Git hub file repository of Phonepe Pulse
02) Cloning the file to local directory in the computer


In [6]:
import subprocess

# Specify the GitHub repository URL
repository_url = "https://github.com/PhonePe/pulse.git"

# Specify the local directory where you want to clone the repository
local_directory ="C:/Users/DIVAHAR/phonepe/DATA"  # Update with your desired directory

# Clone the repository
clone_command = ["git", "clone", repository_url, local_directory]

try:
    subprocess.run(clone_command, check=True)
    print("Repository cloned successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error cloning repository: {e}")

Repository cloned successfully.


# Data Extraction Process

01) Below this we will be extracting 6 different Data from the cloned data 
02) Each extracted Data will be unique different to each other
03) Converting Each extracted data to individual Dataframes
04) Perfoming Data cleaning on each dataframes


# Aggregrated Transaction


In [91]:

# Define a function to clean directory names, including the specific replacement
def clean_directory_name(dir_name):
    dir_name = dir_name.replace("&", "and")  # Replace '&' with 'and'
    dir_name = dir_name.replace("-", " ")    # Replace '-' with space
    return dir_name

path1 = "C:/Users/DIVAHAR/phonepe/DATA/data/aggregated/transaction/country/india/state"
agg_trans_list = os.listdir(path1)

columns1 = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'Transaction_count': [],
            'Transaction_amount': []}

for state in agg_trans_list:
    cleaned_state = clean_directory_name(state)  # Clean the directory name
    cur_state = os.path.join(path1, state)
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = os.path.join(cur_state, year)
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = os.path.join(cur_year, file)
            data = open(cur_file, 'r')
            A = json.load(data)
            
            for i in A['data']['transactionData']:
                name = i['name']
                count = i['paymentInstruments'][0]['count']
                amount = i['paymentInstruments'][0]['amount']
                columns1['Transaction_type'].append(name)
                columns1['Transaction_count'].append(count)
                columns1['Transaction_amount'].append(amount)
                columns1['State'].append(cleaned_state)  # Use the cleaned directory name
                columns1['Year'].append(year)
                columns1['Quarter'].append(int(file.strip('.json')))
                
df_agg_trans = pd.DataFrame(columns1)


In [92]:
df_agg_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3954 entries, 0 to 3953
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               3954 non-null   object 
 1   Year                3954 non-null   object 
 2   Quarter             3954 non-null   int64  
 3   Transaction_type    3954 non-null   object 
 4   Transaction_count   3954 non-null   int64  
 5   Transaction_amount  3954 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 185.5+ KB


In [5]:
df_agg_trans

,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,andaman and nicobar islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman and nicobar islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman and nicobar islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman and nicobar islands,2018,1,Financial Services,33,1.060142e+04
4,andaman and nicobar islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
3949,west bengal,2023,2,Merchant payments,245111000,1.767046e+11
3950,west bengal,2023,2,Peer-to-peer payments,240347041,7.970548e+11
3951,west bengal,2023,2,Recharge & bill payments,58950434,3.478924e+10
3952,west bengal,2023,2,Financial Services,327537,3.174670e+08


In [6]:
import pandas as pd

# Assuming df_agg_trans is your DataFrame
# Use the duplicated() method to identify duplicate rows
duplicate_rows = df_agg_trans[df_agg_trans.duplicated()]

# Check if there are any duplicate rows
if duplicate_rows.shape[0] > 0:
    print("Duplicate rows exist in df_agg_trans.")
    print(duplicate_rows)
else:
    print("No duplicate rows found in df_agg_trans.")

No duplicate rows found in df_agg_trans.


In [7]:
pd.options.display.float_format = '{:.2f}'.format

## OBSERVATION:.

01) Data type of state,quater,Transaction_count,Transaction_type, Transaction_amount are ok and can be remained same 

02) Data type of year from object to int


In [8]:
# To check the data type of state this can be object
df_agg_trans['State'].unique()

array(['andaman and nicobar islands', 'andhra pradesh',
       'arunachal pradesh', 'assam', 'bihar', 'chandigarh',
       'chhattisgarh', 'dadra and nagar haveli and daman and diu',
       'delhi', 'goa', 'gujarat', 'haryana', 'himachal pradesh',
       'jammu and kashmir', 'jharkhand', 'karnataka', 'kerala', 'ladakh',
       'lakshadweep', 'madhya pradesh', 'maharashtra', 'manipur',
       'meghalaya', 'mizoram', 'nagaland', 'odisha', 'puducherry',
       'punjab', 'rajasthan', 'sikkim', 'tamil nadu', 'telangana',
       'tripura', 'uttar pradesh', 'uttarakhand', 'west bengal'],
      dtype=object)

In [9]:
df_agg_trans['Quarter'].unique()

array([1, 2, 3, 4], dtype=int64)

In [10]:
df_agg_trans['Transaction_count'].unique()

array([    4200,     1871,      298, ..., 58950434,   327537,   581674],
      dtype=int64)

In [11]:
df_agg_trans['Transaction_count'].unique()

array(['Recharge & bill payments', 'Peer-to-peer payments',
       'Merchant payments', 'Financial Services', 'Others'], dtype=object)

In [12]:
df_agg_trans['Year'].unique()

array([2018, 2019, 2020, 2021, 2022, 2023])

In [13]:
df_agg_trans['Year'] = df_agg_trans['Year'].astype('int64')


In [14]:
df_agg_trans.head()

,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,andaman and nicobar islands,2018,1,Recharge & bill payments,4200,1845307.47
1,andaman and nicobar islands,2018,1,Peer-to-peer payments,1871,12138655.30
2,andaman and nicobar islands,2018,1,Merchant payments,298,452507.17
3,andaman and nicobar islands,2018,1,Financial Services,33,10601.42
4,andaman and nicobar islands,2018,1,Others,256,184689.87


In [15]:
column_data_types = df_agg_trans.dtypes
print(column_data_types)

State                  object
Year                    int64
Quarter                 int64
Transaction_type       object
Transaction_count       int64
Transaction_amount    float64
dtype: object


In [16]:
df_agg_trans.isnull().sum()

State                 0
Year                  0
Quarter               0
Transaction_type      0
Transaction_count     0
Transaction_amount    0
dtype: int64

# Aggregrated User

In [17]:
import os
import json
import pandas as pd

# Define a function to clean directory names, including the specific replacement
def clean_directory_name(dir_name):
    dir_name = dir_name.replace("&", "and")  # Replace '&' with 'and'
    dir_name = dir_name.replace("-", " ")    # Replace '-' with space
    return dir_name

path2 = "C:\\Users\\DIVAHAR\\phonepe\\DATA\\data\\aggregated\\user\\country\\india\\state"
agg_user_list = os.listdir(path2)

columns2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [],
            'Percentage': []}

for state in agg_user_list:
    cleaned_state = clean_directory_name(state)  # Clean the directory name
    cur_state = os.path.join(path2, state)
    if not os.path.isdir(cur_state):
        continue  # Skip non-directory entries

    agg_year_list = os.listdir(cur_state)

    for year in agg_year_list:
        cleaned_year = clean_directory_name(year)  # Clean the year directory name
        cur_year = os.path.join(cur_state, year)
        if not os.path.isdir(cur_year):
            continue  # Skip non-directory entries

        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = os.path.join(cur_year, file)
            if not os.path.isfile(cur_file):
                continue  # Skip non-file entries

            data = open(cur_file, 'r')
            B = json.load(data)

            try:
                for i in B["data"]["usersByDevice"]:
                    brand_name = i["brand"]
                    counts = i["count"]
                    percents = i["percentage"]
                    columns2["Brands"].append(brand_name)
                    columns2["Count"].append(counts)
                    columns2["Percentage"].append(percents)
                    columns2["State"].append(cleaned_state)
                    columns2["Year"].append(cleaned_year)
                    columns2["Quarter"].append(int(file.strip('.json')))
            except:
                pass

df_agg_user = pd.DataFrame(columns2)


In [18]:
df_agg_user.head()

,State,Year,Quarter,Brands,Count,Percentage
0,andaman and nicobar islands,2018,1,Xiaomi,1665,0.25
1,andaman and nicobar islands,2018,1,Samsung,1445,0.21
2,andaman and nicobar islands,2018,1,Vivo,982,0.15
3,andaman and nicobar islands,2018,1,Oppo,501,0.07
4,andaman and nicobar islands,2018,1,OnePlus,332,0.05


In [19]:
column_data_types = df_agg_user.dtypes
print(column_data_types)

State          object
Year           object
Quarter         int64
Brands         object
Count           int64
Percentage    float64
dtype: object


## Observation :

01) Datatype of year to be changed from object to int

02) Datatype of other columns is ok 


In [20]:
import pandas as pd

# Assuming df_agg_user is your DataFrame
# Use the duplicated() method to identify duplicate rows
duplicate_rows = df_agg_user[df_agg_user.duplicated()]

# Check if there are any duplicate rows
if duplicate_rows.shape[0] > 0:
    print("Duplicate rows exist in df_agg_user.")
    print(duplicate_rows)
else:
    print("No duplicate rows found in df_agg_user.")

No duplicate rows found in df_agg_user.


In [21]:
df_agg_user['Year'] = df_agg_user['Year'].astype('int64')

In [22]:
df_agg_user.head()

,State,Year,Quarter,Brands,Count,Percentage
0,andaman and nicobar islands,2018,1,Xiaomi,1665,0.25
1,andaman and nicobar islands,2018,1,Samsung,1445,0.21
2,andaman and nicobar islands,2018,1,Vivo,982,0.15
3,andaman and nicobar islands,2018,1,Oppo,501,0.07
4,andaman and nicobar islands,2018,1,OnePlus,332,0.05


In [23]:
df_agg_user["State"].unique()

array(['andaman and nicobar islands', 'andhra pradesh',
       'arunachal pradesh', 'assam', 'bihar', 'chandigarh',
       'chhattisgarh', 'dadra and nagar haveli and daman and diu',
       'delhi', 'goa', 'gujarat', 'haryana', 'himachal pradesh',
       'jammu and kashmir', 'jharkhand', 'karnataka', 'kerala', 'ladakh',
       'lakshadweep', 'madhya pradesh', 'maharashtra', 'manipur',
       'meghalaya', 'mizoram', 'nagaland', 'odisha', 'puducherry',
       'punjab', 'rajasthan', 'sikkim', 'tamil nadu', 'telangana',
       'tripura', 'uttar pradesh', 'uttarakhand', 'west bengal'],
      dtype=object)

In [24]:
df_agg_user["Year"].unique()

array([2018, 2019, 2020, 2021, 2022], dtype=int64)

In [25]:
df_agg_user["Quarter"].unique()

array([1, 2, 3, 4], dtype=int64)

In [26]:
df_agg_user["Brands"].unique()

array(['Xiaomi', 'Samsung', 'Vivo', 'Oppo', 'OnePlus', 'Realme', 'Apple',
       'Motorola', 'Lenovo', 'Huawei', 'Others', 'Tecno', 'Gionee',
       'Infinix', 'Asus', 'Micromax', 'HMD Global', 'Lava', 'COOLPAD',
       'Lyf'], dtype=object)

In [27]:
df_agg_user["Count"].dtypes

dtype('int64')

In [28]:
df_agg_user["Percentage"].dtypes

dtype('float64')

In [29]:
df_agg_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6732 entries, 0 to 6731
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   State       6732 non-null   object 
 1   Year        6732 non-null   int64  
 2   Quarter     6732 non-null   int64  
 3   Brands      6732 non-null   object 
 4   Count       6732 non-null   int64  
 5   Percentage  6732 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 315.7+ KB


In [30]:
df_agg_user.isnull().sum()

State         0
Year          0
Quarter       0
Brands        0
Count         0
Percentage    0
dtype: int64

## Map Transaction

In [31]:
import os
import json
import pandas as pd

# Define a function to clean directory names, including the specific replacement
def clean_directory_name(dir_name):
    dir_name = dir_name.replace("&", "and")  # Replace '&' with 'and'
    dir_name = dir_name.replace("-", " ")    # Replace '-' with space
    return dir_name

path3 = "C:\\Users\\DIVAHAR\\phonepe\\DATA\\data\\map\\transaction\\hover\\country\\india\\state"

if not os.path.exists(path3):
    print(f"Directory '{path3}' does not exist.")
else:
    map_trans_list = os.listdir(path3)

    columns3 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Count': [],
                'Amount': []}

    for state in map_trans_list:
        cleaned_state = clean_directory_name(state)  # Clean the directory name
        cur_state = os.path.join(path3, state)
        if not os.path.isdir(cur_state):
            continue  # Skip non-directory entries

        map_year_list = os.listdir(cur_state)

        for year in map_year_list:
            cleaned_year = clean_directory_name(year)  # Clean the year directory name
            cur_year = os.path.join(cur_state, year)
            if not os.path.isdir(cur_year):
                continue  # Skip non-directory entries

            map_file_list = os.listdir(cur_year)

            for file in map_file_list:
                cur_file = os.path.join(cur_year, file)
                if not os.path.isfile(cur_file):
                    continue  # Skip non-file entries

                data = open(cur_file, 'r')
                C = json.load(data)

                try:
                    for i in C["data"]["hoverDataList"]:
                        district = i["name"]
                        count = i["metric"][0]["count"]
                        amount = i["metric"][0]["amount"]
                        columns3["District"].append(district)
                        columns3["Count"].append(count)
                        columns3["Amount"].append(amount)
                        columns3['State'].append(cleaned_state)
                        columns3['Year'].append(cleaned_year)
                        columns3['Quarter'].append(int(file.strip('.json')))
                except:
                    pass

    df_map_trans = pd.DataFrame(columns3)


In [32]:
df_map_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16100 entries, 0 to 16099
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   State     16100 non-null  object 
 1   Year      16100 non-null  object 
 2   Quarter   16100 non-null  int64  
 3   District  16100 non-null  object 
 4   Count     16100 non-null  int64  
 5   Amount    16100 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 754.8+ KB


## Observation :

01) Data type of year to be changed from object to int

02) Other Column datatypes are ok 


In [33]:
import pandas as pd

# Assuming df_top_user is your DataFrame
# Use the duplicated() method to identify duplicate rows
duplicate_rows = df_map_trans[df_map_trans.duplicated()]

# Check if there are any duplicate rows
if duplicate_rows.shape[0] > 0:
    print("Duplicate rows exist in df_map_trans.")
    print(duplicate_rows)
else:
    print("No duplicate rows found in df_map_trans.")

No duplicate rows found in df_map_trans.


In [34]:
df_map_trans["District"].unique()

array(['north and middle andaman district', 'south andaman district',
       'nicobars district', 'east godavari district',
       'srikakulam district', 'spsr nellore district',
       'vizianagaram district', 'visakhapatnam district',
       'prakasam district', 'anantapur district', 'ysr district',
       'west godavari district', 'kurnool district', 'chittoor district',
       'guntur district', 'krishna district',
       'lower dibang valley district', 'lower subansiri district',
       'longding district', 'west siang district', 'kamle district',
       'upper siang district', 'tirap district', 'papum pare district',
       'kurung kumey district', 'siang district', 'lower siang district',
       'kra daadi district', 'changlang district',
       'pakke kessang district', 'dibang valley district',
       'west kameng district', 'east siang district',
       'upper subansiri district', 'east kameng district',
       'lohit district', 'lepa rada district', 'tawang district',
      

In [35]:
df_map_trans["State"].unique()

array(['andaman and nicobar islands', 'andhra pradesh',
       'arunachal pradesh', 'assam', 'bihar', 'chandigarh',
       'chhattisgarh', 'dadra and nagar haveli and daman and diu',
       'delhi', 'goa', 'gujarat', 'haryana', 'himachal pradesh',
       'jammu and kashmir', 'jharkhand', 'karnataka', 'kerala', 'ladakh',
       'lakshadweep', 'madhya pradesh', 'maharashtra', 'manipur',
       'meghalaya', 'mizoram', 'nagaland', 'odisha', 'puducherry',
       'punjab', 'rajasthan', 'sikkim', 'tamil nadu', 'telangana',
       'tripura', 'uttar pradesh', 'uttarakhand', 'west bengal'],
      dtype=object)

In [36]:
df_map_trans["Year"].unique()

array(['2018', '2019', '2020', '2021', '2022', '2023'], dtype=object)

In [37]:
df_map_trans['Year'] = df_map_trans['Year'].astype('int64')

In [38]:
df_map_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16100 entries, 0 to 16099
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   State     16100 non-null  object 
 1   Year      16100 non-null  int64  
 2   Quarter   16100 non-null  int64  
 3   District  16100 non-null  object 
 4   Count     16100 non-null  int64  
 5   Amount    16100 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 754.8+ KB


In [39]:
df_map_trans.isnull().sum()

State       0
Year        0
Quarter     0
District    0
Count       0
Amount      0
dtype: int64

In [40]:
df_map_trans.head()

,State,Year,Quarter,District,Count,Amount
0,andaman and nicobar islands,2018,1,north and middle andaman district,442,931663.08
1,andaman and nicobar islands,2018,1,south andaman district,5688,12560249.34
2,andaman and nicobar islands,2018,1,nicobars district,528,1139848.80
3,andaman and nicobar islands,2018,2,north and middle andaman district,825,1317863.08
4,andaman and nicobar islands,2018,2,south andaman district,9395,23948235.52


In [41]:
df_map_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16100 entries, 0 to 16099
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   State     16100 non-null  object 
 1   Year      16100 non-null  int64  
 2   Quarter   16100 non-null  int64  
 3   District  16100 non-null  object 
 4   Count     16100 non-null  int64  
 5   Amount    16100 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 754.8+ KB


## Map User

In [42]:
import os
import json
import pandas as pd

# Define a function to clean directory names, including the specific replacement
def clean_directory_name(dir_name):
    dir_name = dir_name.replace("&", "and")  # Replace '&' with 'and'
    dir_name = dir_name.replace("-", " ")    # Replace '-' with space
    return dir_name

path4 = "C:\\Users\\DIVAHAR\\phonepe\\DATA\\data\\map\\user\\hover\\country\\india\\state"

if not os.path.exists(path4):
    print(f"Directory '{path4}' does not exist.")
else:
    map_user_list = os.listdir(path4)

    columns4 = {"State": [], "Year": [], "Quarter": [], "District": [],
                "RegisteredUser": [], "AppOpens": []}

    for state in map_user_list:
        cleaned_state = clean_directory_name(state)  # Clean the directory name
        cur_state = os.path.join(path4, state)
        if not os.path.isdir(cur_state):
            continue  # Skip non-directory entries

        map_year_list = os.listdir(cur_state)

        for year in map_year_list:
            cleaned_year = clean_directory_name(year)  # Clean the year directory name
            cur_year = os.path.join(cur_state, year)
            if not os.path.isdir(cur_year):
                continue  # Skip non-directory entries

            map_file_list = os.listdir(cur_year)

            for file in map_file_list:
                cur_file = os.path.join(cur_year, file)
                if not os.path.isfile(cur_file):
                    continue  # Skip non-file entries

                data = open(cur_file, 'r')
                D = json.load(data)

                try:
                    for district, hover_data in D["data"]["hoverData"].items():
                        registered_user = hover_data["registeredUsers"]
                        app_opens = hover_data["appOpens"]
                        columns4["District"].append(district)
                        columns4["RegisteredUser"].append(registered_user)
                        columns4["AppOpens"].append(app_opens)
                        columns4['State'].append(cleaned_state)
                        columns4['Year'].append(cleaned_year)
                        columns4['Quarter'].append(int(file.strip('.json')))
                except:
                    pass

    df_map_user = pd.DataFrame(columns4)


In [43]:
df_map_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16104 entries, 0 to 16103
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   State           16104 non-null  object
 1   Year            16104 non-null  object
 2   Quarter         16104 non-null  int64 
 3   District        16104 non-null  object
 4   RegisteredUser  16104 non-null  int64 
 5   AppOpens        16104 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 755.0+ KB


## Observation :

01) Data type of year to be changed from object to int

02) Other Column datatypes are ok 


In [44]:
import pandas as pd

# Assuming df_top_user is your DataFrame
# Use the duplicated() method to identify duplicate rows
duplicate_rows = df_map_user[df_map_user.duplicated()]

# Check if there are any duplicate rows
if duplicate_rows.shape[0] > 0:
    print("Duplicate rows exist in df_map_user.")
    print(duplicate_rows)
else:
    print("No duplicate rows found in df_map_user.")

No duplicate rows found in df_map_user.


In [45]:
df_map_user.head()

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman and nicobar islands,2018,1,north and middle andaman district,632,0
1,andaman and nicobar islands,2018,1,south andaman district,5846,0
2,andaman and nicobar islands,2018,1,nicobars district,262,0
3,andaman and nicobar islands,2018,2,north and middle andaman district,911,0
4,andaman and nicobar islands,2018,2,south andaman district,8143,0


In [46]:
df_map_user['Year'] = df_map_user['Year'].astype('int64')

In [47]:
df_map_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16104 entries, 0 to 16103
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   State           16104 non-null  object
 1   Year            16104 non-null  int64 
 2   Quarter         16104 non-null  int64 
 3   District        16104 non-null  object
 4   RegisteredUser  16104 non-null  int64 
 5   AppOpens        16104 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 755.0+ KB


In [48]:
df_map_user.isnull().sum()

State             0
Year              0
Quarter           0
District          0
RegisteredUser    0
AppOpens          0
dtype: int64

## Top Transaction



In [49]:
import os
import json
import pandas as pd

# Define a function to clean directory names, including the specific replacement
def clean_directory_name(dir_name):
    dir_name = dir_name.replace("&", "and")  # Replace '&' with 'and'
    dir_name = dir_name.replace("-", " ")    # Replace '-' with space
    return dir_name

path5 = "C:\\Users\\DIVAHAR\\phonepe\\DATA\\data\\top\\transaction\\country\\india\\state"

if not os.path.exists(path5):
    print(f"Directory '{path5}' does not exist.")
else:
    top_trans_list = os.listdir(path5)

    columns5 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'Transaction_count': [],
                'Transaction_amount': []}

    for state in top_trans_list:
        cleaned_state = clean_directory_name(state)  # Clean the directory name
        cur_state = os.path.join(path5, state)
        if not os.path.isdir(cur_state):
            continue  # Skip non-directory entries

        top_year_list = os.listdir(cur_state)

        for year in top_year_list:
            cleaned_year = clean_directory_name(year)  # Clean the year directory name
            cur_year = os.path.join(cur_state, year)
            if not os.path.isdir(cur_year):
                continue  # Skip non-directory entries

            top_file_list = os.listdir(cur_year)

            for file in top_file_list:
                cur_file = os.path.join(cur_year, file)
                if not os.path.isfile(cur_file):
                    continue  # Skip non-file entries

                data = open(cur_file, 'r')
                E = json.load(data)

                try:
                    for i in E['data']['pincodes']:
                        name = i['entityName']
                        count = i['metric']['count']
                        amount = i['metric']['amount']
                        columns5['Pincode'].append(name)
                        columns5['Transaction_count'].append(count)
                        columns5['Transaction_amount'].append(amount)
                        columns5['State'].append(cleaned_state)
                        columns5['Year'].append(cleaned_year)
                        columns5['Quarter'].append(int(file.strip('.json')))
                except:
                    pass

    df_top_trans = pd.DataFrame(columns5)


In [50]:
df_top_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7853 entries, 0 to 7852
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               7853 non-null   object 
 1   Year                7853 non-null   object 
 2   Quarter             7853 non-null   int64  
 3   Pincode             7851 non-null   object 
 4   Transaction_count   7853 non-null   int64  
 5   Transaction_amount  7853 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 368.2+ KB


# Observation : 
01) Year datatype to be changed from object to int 
02) Two missing values are found in Picode column which need to be attended 


In [51]:
import pandas as pd

# Assuming df_top_user is your DataFrame
# Use the duplicated() method to identify duplicate rows
duplicate_rows = df_top_trans[df_top_trans.duplicated()]

# Check if there are any duplicate rows
if duplicate_rows.shape[0] > 0:
    print("Duplicate rows exist in df_top_trans.")
    print(duplicate_rows)
else:
    print("No duplicate rows found in df_top_trans.")

No duplicate rows found in df_top_trans.


In [52]:
df_top_trans.head()

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman and nicobar islands,2018,1,744101,1622,2769297.90
1,andaman and nicobar islands,2018,1,744103,1223,2238041.87
2,andaman and nicobar islands,2018,1,744102,969,3519059.94
3,andaman and nicobar islands,2018,1,744105,685,1298560.95
4,andaman and nicobar islands,2018,1,744104,340,1039715.31


In [3]:
# Converting Data type of year column to integer


In [53]:
df_top_trans['Year'] = df_top_trans['Year'].astype('int64')

In [54]:
pd.options.display.float_format = '{:.2f}'.format

In [55]:
df_top_trans.head()

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,andaman and nicobar islands,2018,1,744101,1622,2769297.90
1,andaman and nicobar islands,2018,1,744103,1223,2238041.87
2,andaman and nicobar islands,2018,1,744102,969,3519059.94
3,andaman and nicobar islands,2018,1,744105,685,1298560.95
4,andaman and nicobar islands,2018,1,744104,340,1039715.31


In [56]:
df_top_trans.shape

(7853, 6)

In [57]:
df_top_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7853 entries, 0 to 7852
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               7853 non-null   object 
 1   Year                7853 non-null   int64  
 2   Quarter             7853 non-null   int64  
 3   Pincode             7851 non-null   object 
 4   Transaction_count   7853 non-null   int64  
 5   Transaction_amount  7853 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 368.2+ KB


In [58]:
df_top_trans.isnull().sum()

State                 0
Year                  0
Quarter               0
Pincode               2
Transaction_count     0
Transaction_amount    0
dtype: int64

In [59]:
df_top_trans['State'].unique()

array(['andaman and nicobar islands', 'andhra pradesh',
       'arunachal pradesh', 'assam', 'bihar', 'chandigarh',
       'chhattisgarh', 'dadra and nagar haveli and daman and diu',
       'delhi', 'goa', 'gujarat', 'haryana', 'himachal pradesh',
       'jammu and kashmir', 'jharkhand', 'karnataka', 'kerala', 'ladakh',
       'lakshadweep', 'madhya pradesh', 'maharashtra', 'manipur',
       'meghalaya', 'mizoram', 'nagaland', 'odisha', 'puducherry',
       'punjab', 'rajasthan', 'sikkim', 'tamil nadu', 'telangana',
       'tripura', 'uttar pradesh', 'uttarakhand', 'west bengal'],
      dtype=object)

In [60]:
df_top_trans['Year'].unique()

array([2018, 2019, 2020, 2021, 2022, 2023], dtype=int64)

In [61]:
df_top_trans['Quarter'].unique()

array([1, 2, 3, 4], dtype=int64)

In [62]:
df_top_trans['Quarter'].unique()

array([1, 2, 3, 4], dtype=int64)

In [4]:
# To see which row has missing Pincode values

In [63]:
rows_with_none_pincode = df_top_trans[df_top_trans['Pincode'].isna()]
rows_with_none_pincode


,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
3794,ladakh,2019,4,None,2014,10098656.17
3836,ladakh,2020,4,None,13717,36711603.92


In [64]:
ladakh_quarter_4_rows = df_top_trans[(df_top_trans['State'] == 'ladakh') & (df_top_trans['Quarter'] == 4)]
ladakh_quarter_4_rows = ladakh_quarter_4_rows.sort_values(by=['Year','Pincode'] )
ladakh_quarter_4_rows

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
3749,ladakh,2018,4,194101,6526,23776399.25
3755,ladakh,2018,4,194102,1799,6137851.00
3752,ladakh,2018,4,194103,2115,7315736.67
3748,ladakh,2018,4,194104,9100,32131162.60
3753,ladakh,2018,4,194105,2051,7670728.19
3756,ladakh,2018,4,194106,1181,5266777.98
3750,ladakh,2018,4,194107,5228,21335611.27
3751,ladakh,2018,4,194201,3905,16456855.18
3757,ladakh,2018,4,194401,693,3798802.39
3754,ladakh,2018,4,194703,1965,9248371.78


In [5]:
# Filling the missing values based on the repeative pincode in ladakh and quater = 4 


In [65]:
df_top_trans['Pincode'].fillna('194105', inplace=True)

In [66]:
df_top_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7853 entries, 0 to 7852
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               7853 non-null   object 
 1   Year                7853 non-null   int64  
 2   Quarter             7853 non-null   int64  
 3   Pincode             7853 non-null   object 
 4   Transaction_count   7853 non-null   int64  
 5   Transaction_amount  7853 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 368.2+ KB


In [67]:
df_top_trans.isnull().sum()

State                 0
Year                  0
Quarter               0
Pincode               0
Transaction_count     0
Transaction_amount    0
dtype: int64

In [68]:

ladakh_quarter_4_rows['Pincode'].unique()

array(['194101', '194102', '194103', '194104', '194105', '194106',
       '194107', '194201', '194401', '194703', '194109', None, '194404'],
      dtype=object)

## Top User

In [69]:
import os
import json
import pandas as pd

# Define a function to clean directory names, including the specific replacement
def clean_directory_name(dir_name):
    dir_name = dir_name.replace("&", "and")  # Replace '&' with 'and'
    dir_name = dir_name.replace("-", " ")    # Replace '-' with space
    return dir_name

# Specify the directory path
path6 = "C:\\Users\\DIVAHAR\\phonepe\\DATA\\data\\top\\user\\country\\india\\state"

if not os.path.exists(path6):
    print(f"Directory '{path6}' does not exist.")
else:
    top_user_list = os.listdir(path6)

    columns6 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'RegisteredUsers': []}

    for state in top_user_list:
        cur_state = os.path.join(path6, state)
        if not os.path.isdir(cur_state):
            continue  # Skip non-directory entries

        top_year_list = os.listdir(cur_state)

        for year in top_year_list:
            cur_year = os.path.join(cur_state, year)
            if not os.path.isdir(cur_year):
                continue  # Skip non-directory entries

            top_file_list = os.listdir(cur_year)

            for file in top_file_list:
                cur_file = os.path.join(cur_year, file)
                if not os.path.isfile(cur_file):
                    continue  # Skip non-file entries

                data = open(cur_file, 'r')
                F = json.load(data)

                try:
                    for i in F['data']['pincodes']:
                        name = i['name']
                        registeredUsers = i['registeredUsers']
                        columns6['Pincode'].append(name)
                        columns6['RegisteredUsers'].append(registeredUsers)
                        columns6['State'].append(state)
                        columns6['Year'].append(year)
                        columns6['Quarter'].append(int(file.strip('.json')))
                except:
                    pass

    df_top_user = pd.DataFrame(columns6)


In [70]:
df_top_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7854 entries, 0 to 7853
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   State            7854 non-null   object
 1   Year             7854 non-null   object
 2   Quarter          7854 non-null   int64 
 3   Pincode          7854 non-null   object
 4   RegisteredUsers  7854 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 306.9+ KB


## Observation:

01) Year and Pincode Data Type to be changed from object to int

02) State,quater,RegisteredUsers  data type is good and can be proceed further

In [71]:
df_top_user.isnull().sum()

State              0
Year               0
Quarter            0
Pincode            0
RegisteredUsers    0
dtype: int64

In [72]:
df_top_user.head()

,State,Year,Quarter,Pincode,RegisteredUsers
0,andaman-&-nicobar-islands,2018,1,744103,1608
1,andaman-&-nicobar-islands,2018,1,744101,1108
2,andaman-&-nicobar-islands,2018,1,744105,1075
3,andaman-&-nicobar-islands,2018,1,744102,1006
4,andaman-&-nicobar-islands,2018,1,744104,272


In [73]:
df_top_user['State'].unique()

array(['andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
       'assam', 'bihar', 'chandigarh', 'chhattisgarh',
       'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
       'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
       'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
       'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
       'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
       'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
       'uttarakhand', 'west-bengal'], dtype=object)

In [74]:
df_top_user['State'] = df_top_user['State'].apply(clean_directory_name)

In [75]:
df_top_user['State'].unique()

array(['andaman and nicobar islands', 'andhra pradesh',
       'arunachal pradesh', 'assam', 'bihar', 'chandigarh',
       'chhattisgarh', 'dadra and nagar haveli and daman and diu',
       'delhi', 'goa', 'gujarat', 'haryana', 'himachal pradesh',
       'jammu and kashmir', 'jharkhand', 'karnataka', 'kerala', 'ladakh',
       'lakshadweep', 'madhya pradesh', 'maharashtra', 'manipur',
       'meghalaya', 'mizoram', 'nagaland', 'odisha', 'puducherry',
       'punjab', 'rajasthan', 'sikkim', 'tamil nadu', 'telangana',
       'tripura', 'uttar pradesh', 'uttarakhand', 'west bengal'],
      dtype=object)

In [6]:
# Coverting Datatype of year and Pincode to int 

In [76]:
df_top_user['Year'] = df_top_user['Year'].astype('int64')
df_top_user['Pincode'] = df_top_user['Pincode'].astype('int64')

In [77]:
df_top_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7854 entries, 0 to 7853
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   State            7854 non-null   object
 1   Year             7854 non-null   int64 
 2   Quarter          7854 non-null   int64 
 3   Pincode          7854 non-null   int64 
 4   RegisteredUsers  7854 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 306.9+ KB


In [78]:
import pandas as pd

# Assuming df_top_user is your DataFrame
# Use the duplicated() method to identify duplicate rows
duplicate_rows = df_top_user[df_top_user.duplicated()]

# Check if there are any duplicate rows
if duplicate_rows.shape[0] > 0:
    print("Duplicate rows exist in df_top_user.")
    print(duplicate_rows)
else:
    print("No duplicate rows found in df_top_user.")


No duplicate rows found in df_top_user.


# Data Transforming Process

## Coverting all data frames to CSV Files

01) Each Extracted Dataframe to be saved a individual csv files
02) This CSV file will later be inserted into MY SQL workbech for visualiation process

In [80]:
# Assuming df_top_user is your DataFrame
# Specify the file path where you want to save the CSV file
csv_file_path = 'df_agg_trans.csv'

# Save the DataFrame to a CSV file
df_agg_trans.to_csv(csv_file_path, index=False)

# Replace 'csv_file_path' with the desired file path where you want to save the CSV file.

In [81]:
# Assuming df_top_user is your DataFrame
# Specify the file path where you want to save the CSV file
csv_file_path = 'df_agg_user.csv'

# Save the DataFrame to a CSV file
df_agg_user.to_csv(csv_file_path, index=False)

# Replace 'csv_file_path' with the desired file path where you want to save the CSV file.

In [82]:
# Assuming df_top_user is your DataFrame
# Specify the file path where you want to save the CSV file
csv_file_path = 'df_map_trans.csv'

# Save the DataFrame to a CSV file
df_map_trans.to_csv(csv_file_path, index=False)

# Replace 'csv_file_path' with the desired file path where you want to save the CSV file.

In [83]:
# Assuming df_top_user is your DataFrame
# Specify the file path where you want to save the CSV file
csv_file_path = 'df_map_user.csv'

# Save the DataFrame to a CSV file
df_map_user.to_csv(csv_file_path, index=False)

# Replace 'csv_file_path' with the desired file path where you want to save the CSV file.

In [84]:
# Assuming df_top_user is your DataFrame
# Specify the file path where you want to save the CSV file
csv_file_path = 'df_top_trans.csv'

# Save the DataFrame to a CSV file
df_top_trans.to_csv(csv_file_path, index=False)

# Replace 'csv_file_path' with the desired file path where you want to save the CSV file.

In [85]:
# Assuming df_top_user is your DataFrame
# Specify the file path where you want to save the CSV file
csv_file_path = 'df_top_user.csv'

# Save the DataFrame to a CSV file
df_top_user.to_csv(csv_file_path, index=False)

# Replace 'csv_file_path' with the desired file path where you want to save the CSV file.


# Connection to  MY SQL Workbench
01) Create a Phonepe database in mysql 
02) Create individual tables with respect to each csv files and the columns data types
03) inserting the vaules in each csv file in their respective tables

In [88]:
import mysql.connector

# Define your MySQL connection parameters
mysql_config = {
    'host': '127.0.0.1',       # Replace with your MySQL host
    'user': 'root',   # Replace with your MySQL username
    'password': '1234' # Replace with your MySQL password
}

# Create a connection to MySQL
connection = mysql.connector.connect(**mysql_config)

# Create a cursor object to execute SQL commands
cursor = connection.cursor()

# Create the "phonepe" database
cursor.execute("CREATE DATABASE IF NOT EXISTS phonepe")

# Close the cursor and MySQL connection
cursor.close()
connection.close()


## df_agg_trans csv to mysql


In [89]:
import pandas as pd
import mysql.connector

# Load the CSV file into a DataFrame
df_agg_trans = pd.read_csv('df_agg_trans.csv')


# Establish a MySQL database connection
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="phonepe" 
)

# Create a cursor
mycursor = mydb.cursor()

# Define the table creation SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS df_agg_trans (
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    Transaction_type VARCHAR(255),
    Transaction_count INT,
    Transaction_amount DOUBLE
)
"""

# Execute the table creation query
mycursor.execute(create_table_query)

# Insert data from the DataFrame into the table
for _, row in df_agg_trans.iterrows():
    insert_query = "INSERT INTO df_agg_trans (State, Year, Quarter, Transaction_type, Transaction_count, Transaction_amount) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['State'], row['Year'], row['Quarter'], row['Transaction_type'], row['Transaction_count'], round(row['Transaction_amount'], 4))
    mycursor.execute(insert_query, values)

# Commit the changes to the database
mydb.commit()

# Close the cursor and the database connection
mycursor.close()
mydb.close()


## df_agg_user csv to mysql


In [94]:
import pandas as pd
import mysql.connector

# Load the CSV file into a DataFrame
df_agg_trans = pd.read_csv('df_agg_user.csv')

# Establish a MySQL database connection
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="phonepe" 
)

# Create a cursor
mycursor = mydb.cursor()

# Define the table creation SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS df_agg_user (
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    Brands VARCHAR(255),
    Count INT,
    Percentage DOUBLE
)
"""

# Execute the table creation query
mycursor.execute(create_table_query)

# Insert data from the DataFrame into the table
for _, row in df_agg_trans.iterrows():
    insert_query = "INSERT INTO df_agg_user (State, Year, Quarter, Brands, Count, Percentage) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['State'], row['Year'], row['Quarter'], row['Brands'], row['Count'], row['Percentage'])
    mycursor.execute(insert_query, values)

# Commit the changes to the database
mydb.commit()

# Close the cursor and the database connection
mycursor.close()
mydb.close()


# map trans csv to mysql

In [95]:
import pandas as pd
import mysql.connector

# Load the CSV file into a DataFrame
df_map_trans = pd.read_csv('df_map_trans.csv')

# Establish a MySQL database connection
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="phonepe"
)

# Create a cursor
mycursor = mydb.cursor()

# Define the table creation SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS df_map_trans (
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    District VARCHAR(255),
    Count INT,
    Amount DOUBLE
)
"""

# Execute the table creation query
mycursor.execute(create_table_query)

# Insert data from the DataFrame into the table
for _, row in df_map_trans.iterrows():
    insert_query = "INSERT INTO df_map_trans (State, Year, Quarter, District, Count, Amount) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['State'], row['Year'], row['Quarter'], row['District'], row['Count'], row['Amount'])
    mycursor.execute(insert_query, values)

# Commit the changes to the database
mydb.commit()

# Close the cursor and the database connection
mycursor.close()
mydb.close()


# map user csv to mysql

In [96]:
import pandas as pd
import mysql.connector

# Load the CSV file into a DataFrame
df_map_user = pd.read_csv('df_map_user.csv')

# Establish a MySQL database connection
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="phonepe"
)

# Create a cursor
mycursor = mydb.cursor()

# Define the table creation SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS df_map_user (
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    District VARCHAR(255),
    RegisteredUser INT,
    AppOpens INT
)
"""

# Execute the table creation query
mycursor.execute(create_table_query)

# Insert data from the DataFrame into the table
for _, row in df_map_user.iterrows():
    insert_query = "INSERT INTO df_map_user (State, Year, Quarter, District, RegisteredUser, AppOpens) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['State'], row['Year'], row['Quarter'], row['District'], row['RegisteredUser'], row['AppOpens'])
    mycursor.execute(insert_query, values)

# Commit the changes to the database
mydb.commit()

# Close the cursor and the database connection
mycursor.close()
mydb.close()


# df_top_tans csv to mysql

In [97]:
import pandas as pd
import mysql.connector

# Load the CSV file into a DataFrame
df_top_trans = pd.read_csv('df_top_trans.csv')

# Establish a MySQL database connection
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="phonepe"
)

# Create a cursor
mycursor = mydb.cursor()

# Define the table creation SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS df_top_trans (
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    Pincode VARCHAR(255),
    Transaction_count INT,
    Transaction_amount DOUBLE
)
"""

# Execute the table creation query
mycursor.execute(create_table_query)

# Insert data from the DataFrame into the table
for _, row in df_top_trans.iterrows():
    insert_query = "INSERT INTO df_top_trans (State, Year, Quarter, Pincode, Transaction_count, Transaction_amount) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['State'], row['Year'], row['Quarter'], row['Pincode'], row['Transaction_count'], round(row['Transaction_amount'], 4))
    mycursor.execute(insert_query, values)

# Commit the changes to the database
mydb.commit()

# Close the cursor and the database connection
mycursor.close()
mydb.close()


# df_top_user csv to mysql

In [98]:
import pandas as pd
import mysql.connector

# Load the CSV file into a DataFrame
df_top_user = pd.read_csv('df_top_user.csv')

# Establish a MySQL database connection
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="phonepe"
)

# Create a cursor
mycursor = mydb.cursor()

# Define the table creation SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS df_top_user (
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    Pincode INT,
    RegisteredUsers INT
)
"""

# Execute the table creation query
mycursor.execute(create_table_query)

# Insert data from the DataFrame into the table
for _, row in df_top_user.iterrows():
    insert_query = "INSERT INTO df_top_user (State, Year, Quarter, Pincode, RegisteredUsers) VALUES (%s, %s, %s, %s, %s)"
    values = (row['State'], row['Year'], row['Quarter'], row['Pincode'], row['RegisteredUsers'])
    mycursor.execute(insert_query, values)

# Commit the changes to the database
mydb.commit()

# Close the cursor and the database connection
mycursor.close()
mydb.close()
